## Is this just magic?  What is Numba doing to make code run quickly?

### Python bytecode

What actually happens when you execute a function written in Python?  It gets parsed and executed by CPython.  
Let's take a trivial example and see what's going on under the hood.

In [ ]:
def add(a, b):
    return a + b

Import the Python disassembler

In [ ]:
import dis

In [ ]:
dis.dis(add)

Makes sense.  Load `a` from the stack, load `b` from the stack, add them together and then return.  Let's see what Numba does in this situation

(Intrigued?  Check out [Playing with Python Bytecode](https://www.youtube.com/watch?v=mxjv9KqzwjI) from PyCon 2016.  It's a great talk!)

In [ ]:
from numba import jit

In [ ]:
@jit
def add(a,b):
    return a + b

In [ ]:
dis.dis(add)

Huh?  It's doing nothing?  

Oh wait, we forgot to run it once so it isn't compiled yet.

In [ ]:
add(1, 1)

In [ ]:
dis.dis(add)

<hr>

Numba has it's own bytecode-ish equivalent.  Once you have run (compiled) `add` you can access the `inspect_types` method.

In [ ]:
add.inspect_types()

Ok.  Numba is doing some LLVM magic, defining things as `int64` and running smoothly.  

[(What happens if you do `add(1., 1.)` and then `inspect_types`?)](./exercises/03.Numba.Types.Demo.ipynb#Numba-type-inference)

## Alternatively

In [ ]:
def add_object(a, b):
    return a.x + b.x

In [ ]:
class MyInt(object):
    def __init__(self, x):
        self.x = x

In [ ]:
a = MyInt(5)
b = MyInt(6)

In [ ]:
add_object(a, b)

In [ ]:
dis.dis(add_object)

In [ ]:
add_object_jit = jit()(add_object)

In [ ]:
add_object_jit(a, b)

In [ ]:
add_object_jit.inspect_types()

## What's all this pyobject business?  

This means it has been compiled in `object` mode.  This can be a faster than regular python if it can do loop lifting, but not that fast.  
We want those `pyobjects` to be `int64` or equivalent type which means forcing `nopython` mode

For the full list of supported Python and NumPy features in `nopython` mode, see the Numba documentation here: http://numba.pydata.org/numba-doc/latest/reference/pysupported.html

## Forcing `nopython` mode

In [ ]:
add_object_jit = jit(nopython=True)(add_object)

In [ ]:
add_object_jit(a, b)

In [ ]:
from numba import njit

In [ ]:
add_object_jit = njit(add_object)

In [ ]:
add_object_jit(a, b)

## Figuring out what isn't working

In [ ]:
%%file nopython_failure.py
from numba import jit

class MyInt(object):
    def __init__(self, x):
        self.x = x
        
@jit
def add_object(a, b):
    for i in range(100):
        c = i
        f = i + 7
        l = c + f
        
    return a.x + b.x

a = MyInt(5)
b = MyInt(6)

add_object(a, b)

In [ ]:
!numba --annotate-html fail.html nopython_failure.py

[fail.html](fail.html)